<a href="https://colab.research.google.com/github/tusharsharm/MDSC-302/blob/main/Deep_Learning%20%7C%20MDSC302%20%7C%2024040208008%20AS_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDSC 302 | Deep Learning Lab  | Assignment 1

# **Constructing a tensor**
## Tensors are the building blocks for representing data in PyTorch.


### Imports the torch module

In [1]:
import torch

### Create a one-dimensional tensor of size 3 filled with 1s

In [2]:
a = torch.ones(10)

In [3]:
a

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [4]:
a[1]

tensor(1.)

In [5]:
float(a[1])

1.0

In [6]:
a[2] = 2.0

In [7]:
a

tensor([1., 1., 2., 1., 1., 1., 1., 1., 1., 1.])

# Indexing tensors

- Using .zeros is just a way to get
an appropriately sized array.
- We overwrite those zeros with
the values we actually want.

In [8]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 8.0
points[4] = 9.0
points[5] = 1.0


### We can also pass a Python list to the constructor, to the same effect:

In [9]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

### To get the coordinates of the first point, we do the following:

In [10]:
float(points[0]), float(points[1])

(4.0, 1.0)

### we can use a 2D tensor:

In [11]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

###We can ask the tensor about its shape:

In [12]:
points.shape

torch.Size([3, 2])

- This informs us about the size of the tensor along each dimension.
- We could also use
zeros or ones to initialize the tensor, providing the size as a tuple:

In [13]:
points = torch.ones(3, 2)
points

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])

###we can access an individual element in the tensor using two indices:

In [14]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

# Named Tensors

## Named tensors  allow you to assign meaningful names to the dimensions of a tensor, rather than just referring to them by their numerical index.

In [15]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [16]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [17]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

## PyTorch will allow us to multiply things that are the same shape, as well as shapes where one operand is of size 1 in a given dimension.
##It also appends leading dimensions of size 1 automatically. This is a feature called **broadcasting**.

In [18]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [19]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [20]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/tmp/ipython-input-20-2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1935.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

- When we already have a tensor and want to add names (but not change existing ones), we can call the method refine_names on it

In [21]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


- align_as returns a tensor with missing dimensions
added and existing ones permuted to the right order:

In [22]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [23]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [24]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

# Managing a tensor's dtype attribute
### Managing a tensor's dtype attribute involves understanding how to inspect, specify, and convert the data type of a tensor.

In [25]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [26]:
short_points.dtype

torch.int16

In [27]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [28]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)


- rand initializes the tensor elements to
random numbers between 0 and 1.

In [29]:
points_64 = torch.rand(5, dtype=torch.double)
points_short = points_64.to(torch.short)


points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

# The tensor API
### a set of programming interfaces used to work with tensors, which are multi-dimensional arrays

In [30]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

In [31]:
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [32]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

# Indexing into storage

### The storage for a given tensor is accessible using the .storage property:

In [33]:
points = torch.tensor([[4.0,1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

/tmp/ipython-input-33-3168638266.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

- Even though the tensor reports itself as having three rows and two columns, the storage under the hood is a contiguous array of size 6.
- In this sense, the tensor just knows how to translate a pair of indices into a location in the storage.
- We can also index into a storage manually. For instance:

In [34]:
points_storage = points.storage()
points_storage[0]

4.0

### We can’t index a storage of a 2D tensor using two indices. The layout of a storage is always one-dimensional, regardless of the dimensionality of any and all tensors that might refer to it.

In [35]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

# Modifying stored values: In-place operations

- In addition to the operations on tensors introduced in the previous section, a small number of operations exist only as methods of the Tensor object. They are recognizable from a trailing underscore in their name, like zero_, which indicates that the method operates in place by modifying the input instead of creating a new output tensor and returning it.
-  For instance, the zero_ method zeros out all the elements of the input.
- Any method without the trailing underscore leaves the source tensor unchanged and instead returns a new tensor:

In [36]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

# Tensor metadata: Size, offset, and stride

- Size (or Shape): Defines the logical dimensions of the tensor (e.g., how many rows and columns a matrix has).
- Offset: Specifies the starting point of the tensor's data within the underlying, contiguous memory block or storage.
- Stride: Indicates how many elements to skip in the storage to access the next element along each dimension of the tensor. For instance, a stride of (2,1) for a 2x3 tensor suggests that to move to the next row, one needs to skip 2 elements in memory, while to move to the next column, only 1 element needs to be skipped.

## Views of another tensor’s storage

In [37]:
#We can get the second point in the tensor by providing the corresponding index:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [38]:
second_point.size()

torch.Size([2])

In [39]:
second_point.shape


torch.Size([2])

In [40]:
points.stride()

(2, 1)

In [41]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [42]:
second_point.storage_offset()

2

In [43]:
second_point.stride()

(1,)

###subtensor has one less dimension, as we would expect,while still indexing the same storage as the original points tensor.
###This also means changing the subtensor will have a side effect on the original tensor:

In [44]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [45]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

## Transposing without copying

In [46]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [47]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

###We can easily verify that the two tensors share the same storage

In [48]:
id(points.storage()) == id(points_t.storage())

True

###and that they differ only in shape and stride:

In [49]:
x = points.stride()
y = points_t.stride()
print(x,y)

(2, 1) (1, 2)


## Transposing in higher dimensions
### Transposing in PyTorch is not limited to matrices. We can transpose a multidimensional array by specifying the two dimensions along which transposing (flipping shape and stride) should occur:

In [50]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

torch.Size([3, 4, 5])

In [51]:
transpose_t.shape

torch.Size([5, 4, 3])

In [52]:
torch.Size([5, 4, 3])

torch.Size([5, 4, 3])

In [53]:
some_t.stride()


(20, 5, 1)

In [54]:
transpose_t.stride()

(1, 5, 20)

## Contiguous tensors
- Some tensor operations in PyTorch only work on contiguous tensors, such as *view*

In [55]:
points.is_contiguous()

True

In [56]:
points_t.is_contiguous()

False

###We can obtain a new contiguous tensor from a non-contiguous one using the contiguous method. The content of the tensor will be the same, but the stride will change, as will the storage:

In [57]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [58]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [59]:
points_t.stride()

(1, 2)

In [60]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [61]:
points_t_cont.stride()

(3, 1)

In [62]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

# NumPy interoperability

### PyTorch tensors can be converted to NumPy arrays and vice versa very efficiently.By doing so, we can take advantage of the huge swath of functionality in the wider Python ecosystem that has built up around the NumPy array type. This zero-copy interoperability with NumPy arrays is due to the storage system working with the Python buffer protocol

In [63]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

###Conversely, we can obtain a PyTorch tensor from a NumPy array this way

In [64]:
points = torch.from_numpy(points_np)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])